In [ ]:
import os, shutil
import numpy as np
from sklearn.model_selection import train_test_split

# Paths
src_dir = "/content/dataset"  # where 0/ and 1/ are currently
base_dir = "/content/split_dataset"

# Make directories
for split in ["train", "val", "test"]:
    for label in ["0", "1"]:
        os.makedirs(os.path.join(base_dir, split, label), exist_ok=True)

def split_and_copy(class_name):
    imgs = os.listdir(os.path.join(src_dir, class_name))
    imgs = [f for f in imgs if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    train_imgs, temp_imgs = train_test_split(imgs, test_size=0.3, random_state=42)
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

    for img in train_imgs:
        shutil.copy(os.path.join(src_dir, class_name, img),
                    os.path.join(base_dir, "train", class_name, img))
    for img in val_imgs:
        shutil.copy(os.path.join(src_dir, class_name, img),
                    os.path.join(base_dir, "val", class_name, img))
    for img in test_imgs:
        shutil.copy(os.path.join(src_dir, class_name, img),
                    os.path.join(base_dir, "test", class_name, img))

split_and_copy("0")
split_and_copy("1")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

train_dir = r'C:\Users\tarun\dataset\training'
val_dir = r'C:\Users\tarun\dataset\validation'
test_dir = r'C:\users\tarun\dataset\training'

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir,
                                              target_size=(128,128),
                                              batch_size=32,
                                              class_mode='binary')

val_gen = val_datagen.flow_from_directory(val_dir,
                                          target_size=(128,128),
                                          batch_size=32,
                                          class_mode='binary')

test_gen = val_datagen.flow_from_directory(test_dir,
                                           target_size=(128,128),
                                           batch_size=32,
                                           class_mode='binary',
                                           shuffle=False)

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.3),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(train_gen,
                    validation_data=val_gen,
                    epochs=50,
                    callbacks=[early_stop],
                    verbose=1)

loss, acc = model.evaluate(test_gen, verbose=0)
print(f"Test Accuracy: {acc*100:.2f}%")

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy over Epochs')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss over Epochs')

plt.tight_layout()
plt.show()
